In [1]:
# Add this at the very start of your notebook:
from google.colab import drive
drive.mount('/content/drive')

# Verify mount worked
import os
if not os.path.exists('/content/drive/MyDrive'):
    raise Exception("❌ Google Drive not mounted! Please run: drive.mount('/content/drive')")

Mounted at /content/drive


In [ ]:
!nvidia-smi

Thu Jun 12 01:05:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   38C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch

if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")


Using GPU: NVIDIA L4


# CIFAR-100 Architecture Comparison

### Setup Drive Base Path

In [3]:
DRIVE_BASE_PATH = '/content/drive/MyDrive/CIFAR100_Experiment'

def setup_drive_directories():
    """Create necessary directories in Google Drive"""
    drive_dirs = [
        DRIVE_BASE_PATH,
        f"{DRIVE_BASE_PATH}/models",
        f"{DRIVE_BASE_PATH}/results",
        f"{DRIVE_BASE_PATH}/progress",
        f"{DRIVE_BASE_PATH}/plots"
    ]

    for dir_path in drive_dirs:
        os.makedirs(dir_path, exist_ok=True)

    print(f"📁 Drive directories created at: {DRIVE_BASE_PATH}")
    return DRIVE_BASE_PATH

1. Install required packages

In [4]:
# !pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install -q timm  # For modern model architectures
# !pip install -q matplotlib seaborn pandas scikit-learn
# !pip install -q tqdm

2. Import essential libraries

In [5]:
import os
import sys
import time
import traceback
from pathlib import Path
import json
import pickle
import gc

import warnings
warnings.filterwarnings('ignore')

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.ioff()

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

### COMPLETE MODEL DEFINITIONS - ALL MODELS FROM ORIGINAL CODE

3. BASELINE MODELS


- SimpleCNN
- LeNet
- AlexNet

In [6]:
class SimpleCNN(nn.Module):
    """Simple CNN with 3 convolutional layers - Original baseline"""

    def __init__(self, num_classes=100):
        super(SimpleCNN, self).__init__()

        self.features = nn.Sequential(
            # First conv block
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Second conv block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Third conv block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((7, 7)),
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [7]:
class LeNet(nn.Module):
    """LeNet architecture adapted for RGB images - Original baseline"""

    def __init__(self, num_classes=100):
        super(LeNet, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16 * 53 * 53, 120),
            nn.ReLU(inplace=True),
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            nn.Linear(84, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [8]:
class AlexNet(nn.Module):
    """AlexNet architecture - Original baseline"""

    def __init__(self, num_classes=100, dropout=0.5):
        super(AlexNet, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

4. MODERN CNN ARCHITECTURES

- ResNet18
- ResNet50
- EfficientNet_B0
- EfficientNet_B2
- RegNet-Y-400MF

In [9]:
def get_resnet18(num_classes=100, pretrained=True, **kwargs):
    """ResNet-18 with residual connections"""
    if pretrained:
        model = models.resnet18(weights='DEFAULT')
    else:
        model = models.resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

def get_resnet50(num_classes=100, pretrained=True, **kwargs):
    """ResNet-50 with deeper residual connections"""
    if pretrained:
        model = models.resnet50(weights='DEFAULT')
    else:
        model = models.resnet50(weights=None)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

def get_efficientnet_b0(num_classes=100, pretrained=True, dropout_rate=0.2, **kwargs):
    """EfficientNet-B0 - Compound scaling for efficiency"""
    if pretrained:
        model = models.efficientnet_b0(weights='DEFAULT')
    else:
        model = models.efficientnet_b0(weights=None)
    model.classifier = nn.Sequential(
        nn.Dropout(p=dropout_rate, inplace=True),
        nn.Linear(model.classifier[1].in_features, num_classes)
    )
    return model

def get_efficientnet_b2(num_classes=100, pretrained=True, dropout_rate=0.3, **kwargs):
    """EfficientNet-B2 - Larger compound scaled version"""
    if pretrained:
        model = models.efficientnet_b2(weights='DEFAULT')
    else:
        model = models.efficientnet_b2(weights=None)
    model.classifier = nn.Sequential(
        nn.Dropout(p=dropout_rate, inplace=True),
        nn.Linear(model.classifier[1].in_features, num_classes)
    )
    return model

def get_regnet_y_400mf(num_classes=100, pretrained=True, **kwargs):
    """RegNet-Y-400MF - Efficient design space exploration"""
    if pretrained:
        model = models.regnet_y_400mf(weights='DEFAULT')
    else:
        model = models.regnet_y_400mf(weights=None)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

5. Vision Transformers

- Vit-Base-16
- Deit-Small
- Swin-Tiny

In [10]:
def get_vit_base_16(num_classes=100, pretrained=True, **kwargs):
    """Vision Transformer Base/16 - Pure transformer for images"""
    try:
        import timm
        model = timm.create_model('vit_base_patch16_224', pretrained=pretrained, num_classes=num_classes)
        return model
    except Exception as e:
        print(f"⚠️ Error loading ViT: {e}")
        # Fallback to ResNet18
        return get_resnet18(num_classes, pretrained)

def get_deit_small(num_classes=100, pretrained=True, **kwargs):
    """Data-efficient Image Transformer Small"""
    try:
        import timm
        model = timm.create_model('deit_small_patch16_224', pretrained=pretrained, num_classes=num_classes)
        return model
    except Exception as e:
        print(f"⚠️ Error loading DeiT: {e}")
        # Fallback to ResNet18
        return get_resnet18(num_classes, pretrained)

def get_swin_tiny(num_classes=100, pretrained=True, **kwargs):
    """Swin Transformer Tiny - Hierarchical vision transformer"""
    try:
        import timm
        model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=pretrained, num_classes=num_classes)
        return model
    except Exception as e:
        print(f"⚠️ Error loading Swin: {e}")
        # Fallback to ResNet18
        return get_resnet18(num_classes, pretrained)

5. Mobile Optimized Model

- MobileNet-v3-Small
- EfficientNet-Lite0

In [11]:
def get_mobilenet_v3_small(num_classes=100, pretrained=True, **kwargs):
    """MobileNet V3 Small - Optimized for mobile deployment"""
    if pretrained:
        model = models.mobilenet_v3_small(weights='DEFAULT')
    else:
        model = models.mobilenet_v3_small(weights=None)

    model.classifier = nn.Sequential(
        nn.Linear(model.classifier[0].in_features, model.classifier[0].out_features),
        nn.Hardswish(inplace=True),
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(model.classifier[3].in_features, num_classes)
    )
    return model

def get_efficientnet_lite0(num_classes=100, pretrained=True, **kwargs):
    """EfficientNet-Lite0 - Mobile-optimized variant"""
    try:
        import timm
        model = timm.create_model('efficientnet_lite0', pretrained=pretrained, num_classes=num_classes)
        return model
    except Exception as e:
        print(f"⚠️ Error loading EfficientNet-Lite: {e}")
        # Fallback to MobileNet
        return get_mobilenet_v3_small(num_classes, pretrained)

### Models Registry

In [12]:
MODEL_REGISTRY = {
    # Original baselines
    'simple_cnn': SimpleCNN,
    'lenet': LeNet,
    'alexnet': AlexNet,

    # Modern CNNs
    'resnet18': get_resnet18,
    'resnet50': get_resnet50,
    'efficientnet_b0': get_efficientnet_b0,
    'efficientnet_b2': get_efficientnet_b2,
    'regnet_y_400mf': get_regnet_y_400mf,

    # Vision Transformers
    'vit_base_16': get_vit_base_16,
    'deit_small': get_deit_small,
    'swin_tiny': get_swin_tiny,

    # Mobile models
    'mobilenet_v3_small': get_mobilenet_v3_small,
    'efficientnet_lite0': get_efficientnet_lite0
}

def get_model(model_name: str, num_classes: int = 100, **kwargs) -> nn.Module:
    """Factory function to get models by name"""

    if model_name not in MODEL_REGISTRY:
        available_models = list(MODEL_REGISTRY.keys())
        raise ValueError(f"Unknown model: {model_name}. Available models: {available_models}")

    # Get model
    model_fn = MODEL_REGISTRY[model_name]

    try:
        if model_name in ['simple_cnn', 'lenet', 'alexnet']:
            # Original models without pretrained weights
            model = model_fn(num_classes=num_classes)
        else:
            # Modern models with pretrained weights
            model = model_fn(num_classes=num_classes, **kwargs)

        return model

    except Exception as e:
        print(f"Error creating model {model_name}: {e}")
        print("Falling back to SimpleCNN...")
        model = SimpleCNN(num_classes=num_classes)
        return model

### Trainer for all Models

In [13]:
class ColabTrainer:
    """Enhanced trainer that SAVES model checkpoints"""

    def __init__(self, model, device='cuda', learning_rate=0.001, weight_decay=1e-4, model_name="model"):
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device)
        self.model_name = model_name

        # Create models directory
        setup_drive_directories()
        self.models_dir = Path(f'{DRIVE_BASE_PATH}/models')
        self.models_dir.mkdir(parents=True, exist_ok=True)

        # Optimizer and loss with weight decay
        self.criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
        self.optimizer = optim.AdamW(
            self.model.parameters(),
            lr=learning_rate,
            weight_decay=weight_decay,
            betas=(0.9, 0.999)
        )

        # Scheduler
        self.scheduler = optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=20)

        # Mixed precision if available
        self.use_mixed_precision = torch.cuda.is_available()
        if self.use_mixed_precision:
            try:
                self.scaler = torch.cuda.amp.GradScaler()
            except:
                self.use_mixed_precision = False
                self.scaler = None
        else:
            self.scaler = None

        # Training history
        self.train_history = {'loss': [], 'accuracy': [], 'learning_rate': []}
        self.test_history = {'loss': [], 'accuracy': []}
        self.best_accuracy = 0.0

    def save_checkpoint(self, epoch, is_best=False):
        """Save model checkpoint"""
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_accuracy': self.best_accuracy,
            'train_history': self.train_history,
            'test_history': self.test_history,
            'model_name': self.model_name
        }

        if self.scaler:
            checkpoint['scaler_state_dict'] = self.scaler.state_dict()

        # Save regular checkpoint
        checkpoint_path = self.models_dir / f'{self.model_name}_checkpoint.pth'
        torch.save(checkpoint, checkpoint_path)

        # Save best model separately
        if is_best:
            best_path = self.models_dir / f'{self.model_name}_best.pth'
            torch.save(checkpoint, best_path)
            print(f"💾 Best model saved: {best_path}")

        print(f"💾 Checkpoint saved: {checkpoint_path}")

    def load_checkpoint(self, checkpoint_path):
        """Load model checkpoint if it exists"""
        drive_checkpoint_path = self.models_dir / f'{self.model_name}_checkpoint.pth'

        if drive_checkpoint_path.exists():
            print(f"📂 Loading checkpoint from Drive: {drive_checkpoint_path}")
            checkpoint = torch.load(drive_checkpoint_path, map_location=self.device)

            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            self.best_accuracy = checkpoint['best_accuracy']
            self.train_history = checkpoint['train_history']
            self.test_history = checkpoint['test_history']

            if self.scaler and 'scaler_state_dict' in checkpoint:
                self.scaler.load_state_dict(checkpoint['scaler_state_dict'])

            return checkpoint['epoch']
        return 0

    def train_epoch(self, train_loader, epoch):
        """Train for one epoch with mixed precision"""
        self.model.train()
        total_loss = 0.0
        correct = 0
        total = 0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")

        for batch_idx, (data, target) in enumerate(progress_bar):
            data, target = data.to(self.device), target.to(self.device)

            self.optimizer.zero_grad()

            # Mixed precision forward pass
            if self.use_mixed_precision:
                with torch.cuda.amp.autocast():
                    output = self.model(data)
                    loss = self.criterion(output, target)

                self.scaler.scale(loss).backward()
                self.scaler.step(self.optimizer)
                self.scaler.update()
            else:
                output = self.model(data)
                loss = self.criterion(output, target)
                loss.backward()
                self.optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            # Update progress bar
            current_lr = self.optimizer.param_groups[0]['lr']
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc': f'{100.*correct/total:.2f}%',
                'LR': f'{current_lr:.2e}'
            })

        epoch_loss = total_loss / len(train_loader)
        epoch_acc = correct / total

        return epoch_loss, epoch_acc

    def evaluate(self, test_loader):
        """Evaluate model on test set"""
        self.model.eval()
        total_loss = 0.0
        correct = 0
        total = 0
        all_predictions = []
        all_targets = []

        with torch.no_grad():
            for data, target in tqdm(test_loader, desc="Evaluating"):
                data, target = data.to(self.device), target.to(self.device)

                if self.use_mixed_precision:
                    with torch.cuda.amp.autocast():
                        output = self.model(data)
                else:
                    output = self.model(data)

                loss = self.criterion(output, target)
                total_loss += loss.item()

                _, predicted = torch.max(output, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

                all_predictions.extend(predicted.cpu().numpy())
                all_targets.extend(target.cpu().numpy())

        test_loss = total_loss / len(test_loader)
        test_acc = correct / total

        return test_loss, test_acc, all_predictions, all_targets

    def train(self, train_loader, test_loader, epochs=15, early_stopping_patience=5, save_every=5):
        """Complete training loop with checkpointing"""
        print(f"🚀 Training {self.model_name} on {self.device}")
        print(f"Mixed Precision: {self.use_mixed_precision}")
        print(f"💾 Checkpoints will be saved to: {self.models_dir}")

        # Try to load existing checkpoint
        start_epoch = self.load_checkpoint(self.models_dir / f'{self.model_name}_checkpoint.pth')

        patience_counter = 0
        start_time = time.time()

        predictions = []
        targets = []

        for epoch in range(start_epoch, epochs):
            # Training
            train_loss, train_acc = self.train_epoch(train_loader, epoch)

            # Evaluation
            test_loss, test_acc, predictions, targets = self.evaluate(test_loader)

            # Update scheduler
            self.scheduler.step()
            current_lr = self.optimizer.param_groups[0]['lr']

            # Save history
            self.train_history['loss'].append(train_loss)
            self.train_history['accuracy'].append(train_acc)
            self.train_history['learning_rate'].append(current_lr)
            self.test_history['loss'].append(test_loss)
            self.test_history['accuracy'].append(test_acc)

            # Check if best model
            is_best = test_acc > self.best_accuracy
            if is_best:
                self.best_accuracy = test_acc
                patience_counter = 0
            else:
                patience_counter += 1

            print(f"Epoch [{epoch+1}/{epochs}]")
            print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
            print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
            print(f"Best Acc: {self.best_accuracy:.4f}, Patience: {patience_counter}/{early_stopping_patience}")

            # Save checkpoint every N epochs or if best
            # Always save best model immediately
            if is_best:
                self.save_checkpoint(epoch + 1, is_best=True)

            # Save regular checkpoint every save_every epochs
            if (epoch + 1) % save_every == 0:
                self.save_checkpoint(epoch + 1, is_best=False)

            # if (epoch + 1) % save_every == 0 or is_best:
            #     self.save_checkpoint(epoch + 1, is_best)

            print("-" * 60)

            # Early stopping
            if patience_counter >= early_stopping_patience:
                print(f"🛑 Early stopping triggered after {epoch+1} epochs")
                break

        # Save final checkpoint
        self.save_checkpoint(epochs, False)

        training_time = time.time() - start_time

        return {
            'final_test_acc': self.best_accuracy,
            'training_time': training_time,
            'train_history': self.train_history,
            'test_history': self.test_history,
            'predictions': predictions,
            'targets': targets
        }

### CIFAR-100 Data Loading

In [14]:
def get_cifar100_dataloaders(batch_size=64, num_workers=2):
    """Create CIFAR-100 data loaders optimized for Colab"""

    # Enhanced data transforms
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5071, 0.4867, 0.4408],
                           std=[0.2675, 0.2565, 0.2761])
    ])

    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5071, 0.4867, 0.4408],
                           std=[0.2675, 0.2565, 0.2761])
    ])

    # Download and create datasets
    train_dataset = datasets.CIFAR100(
        root='./data', train=True, download=True, transform=train_transform
    )
    test_dataset = datasets.CIFAR100(
        root='./data', train=False, download=True, transform=test_transform
    )

    # Create dataloaders
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, pin_memory=True
    )
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=True
    )

    # CIFAR-100 class names
    class_names = [
        'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle',
        'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel',
        'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock',
        'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur',
        'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster',
        'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
        'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
        'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
        'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
        'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
        'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
        'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
        'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
        'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
        'worm'
    ]

    print(f"✅ CIFAR-100 loaded successfully!")
    print(f"   Training samples: {len(train_dataset):,}")
    print(f"   Test samples: {len(test_dataset):,}")
    print(f"   Classes: {len(class_names)}")

    return train_loader, test_loader, class_names

### LOGISTIC REGRESSION BASELINE

In [15]:
def train_logistic_regression_baseline(train_loader, test_loader):
    """Train logistic regression baseline for comparison"""
    print("📊 Training Logistic Regression Baseline...")

    # Extract features and labels
    X_train, y_train = [], []
    X_test, y_test = [], []

    # Simple transform for logistic regression
    simple_transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Recreate datasets with simple transform
    train_dataset_lr = datasets.CIFAR100(root='./data', train=True, transform=simple_transform)
    test_dataset_lr = datasets.CIFAR100(root='./data', train=False, transform=simple_transform)

    train_loader_lr = DataLoader(train_dataset_lr, batch_size=1000, shuffle=False)
    test_loader_lr = DataLoader(test_dataset_lr, batch_size=1000, shuffle=False)

    # Convert to flattened arrays
    print("Converting training data...")
    for data, target in tqdm(train_loader_lr, desc="Loading train data"):
        X_train.append(data.numpy().reshape(data.size(0), -1))
        y_train.append(target.numpy())

    print("Converting test data...")
    for data, target in tqdm(test_loader_lr, desc="Loading test data"):
        X_test.append(data.numpy().reshape(data.size(0), -1))
        y_test.append(target.numpy())

    X_train = np.vstack(X_train)
    y_train = np.hstack(y_train)
    X_test = np.vstack(X_test)
    y_test = np.hstack(y_test)

    print(f"Training data shape: {X_train.shape}")
    print(f"Test data shape: {X_test.shape}")

    # Train logistic regression
    print("Normalizing features...")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    print("Fitting logistic regression model...")
    model = LogisticRegression(
        max_iter=1000,
        random_state=42,
        multi_class='ovr'
    )

    start_time = time.time()
    model.fit(X_train_scaled, y_train)
    training_time = time.time() - start_time

    # Evaluate
    print("Making predictions...")
    train_predictions = model.predict(X_train_scaled)
    test_predictions = model.predict(X_test_scaled)

    train_accuracy = accuracy_score(y_train, train_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)

    # Calculate additional metrics
    # precision, recall, f1, _ = precision_recall_fscore_support(
    #     y_test, test_predictions, average='weighted'
    # )

    print(f"✅ Logistic Regression Results:")
    print(f"   Training Accuracy: {train_accuracy*100:.2f}%")
    print(f"   Test Accuracy: {test_accuracy*100:.2f}%")
    # print(f"   Precision: {precision:.4f}")
    # print(f"   Recall: {recall:.4f}")
    # print(f"   F1-Score: {f1:.4f}")
    print(f"   Training Time: {training_time:.2f}s")

    return {
        'final_test_acc': test_accuracy,
        'training_time': training_time,
        'model_name': 'Logistic Regression'
    }

### Training Configuration

In [16]:
def get_training_config(model_name: str, device: str) -> dict:
    """Get optimized training configuration for each model type"""

    # Base configuration
    base_config = {
        'device': device,
        'learning_rate': 0.001,
        'weight_decay': 1e-4,
        'epochs': 15,
        'early_stopping_patience': 5,
        'batch_size': 64
    }

    # Model-specific adjustments
    if model_name in ['alexnet', 'resnet50', 'efficientnet_b2']:
        # Larger models - reduce batch size, slightly more epochs
        base_config.update({
            'batch_size': 32,
            'epochs': 20,
            'learning_rate': 0.0008
        })

    elif model_name in ['vit_base_16', 'deit_small']:  #, 'swin_tiny']:
        # Transformers - lower learning rate, smaller batch size
        base_config.update({
            'batch_size': 24,
            'learning_rate': 0.0005,
            'epochs': 18,
            'early_stopping_patience': 7
        })

    elif model_name in ['mobilenet_v3_small', 'efficientnet_lite0']:
        # Mobile models - can handle larger batch size
        base_config.update({
            'batch_size': 96,
            'learning_rate': 0.0012
        })

    elif model_name in ['simple_cnn', 'lenet']:
        # Simple models - faster training
        base_config.update({
            'epochs': 25,
            'learning_rate': 0.002
        })

    elif model_name == 'swin_tiny':
        # Special config for Swin Tiny (it performed terribly before)
        base_config.update({
            'batch_size': 16,           # Smaller batch size
            'learning_rate': 0.0005,    # Much lower learning rate
            'scheduler': 'cosine',
            'pretrained': True,
            'epochs': 25,               # More epochs
            'early_stopping_patience': 10,  # More patience
            'weight_decay': 0.05        # Higher weight decay
        })

    return base_config

### Enhancedd Visualization

In [17]:
def plot_comprehensive_results(results_dict):
    """Create comprehensive visualization of all results"""

    # Filter out models that failed
    valid_results = {k: v for k, v in results_dict.items() if v is not None}

    if len(valid_results) == 0:
        print("No valid results to plot")
        return

    fig = plt.figure(figsize=(20, 16))

    # 1. Performance Ranking (Top Left)
    ax1 = plt.subplot(2, 3, 1)
    models = list(valid_results.keys())
    accuracies = [valid_results[m]['final_test_acc']*100 for m in models]

    # Sort by accuracy
    sorted_indices = np.argsort(accuracies)
    sorted_models = [models[i] for i in sorted_indices]
    sorted_accuracies = [accuracies[i] for i in sorted_indices]

    colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(sorted_models)))
    bars = ax1.barh(range(len(sorted_models)), sorted_accuracies, color=colors, alpha=0.8)

    # Add accuracy labels
    for i, (bar, acc) in enumerate(zip(bars, sorted_accuracies)):
        ax1.text(acc + 0.5, bar.get_y() + bar.get_height()/2,
                f'{acc:.1f}%', va='center', fontweight='bold')

    ax1.set_yticks(range(len(sorted_models)))
    ax1.set_yticklabels([m.replace('_', ' ').title() for m in sorted_models])
    ax1.set_xlabel('Test Accuracy (%)')
    ax1.set_title('Model Performance Ranking', fontweight='bold', fontsize=14)
    ax1.grid(axis='x', alpha=0.3)

    # 2. Training Time Comparison (Top Middle)
    ax2 = plt.subplot(2, 3, 2)
    training_times = [valid_results[m]['training_time']/60 for m in models]  # Convert to minutes

    bars = ax2.bar(range(len(models)), training_times, alpha=0.7, color='orange')
    for bar, time_min in zip(bars, training_times):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(training_times)*0.01,
                f'{time_min:.1f}m', ha='center', fontweight='bold', fontsize=9)

    ax2.set_xticks(range(len(models)))
    ax2.set_xticklabels([m.replace('_', ' ').title() for m in models], rotation=45, ha='right')
    ax2.set_ylabel('Training Time (minutes)')
    ax2.set_title('Training Time Comparison', fontweight='bold', fontsize=14)
    ax2.grid(axis='y', alpha=0.3)

    # 3. Efficiency Analysis (Top Right)
    ax3 = plt.subplot(2, 3, 3)
    efficiency = [acc/time for acc, time in zip(accuracies, training_times)]

    scatter = ax3.scatter(training_times, accuracies,
                         s=[100 + e*10 for e in efficiency],
                         alpha=0.6, c=range(len(models)), cmap='plasma')

    for i, model in enumerate(models):
        ax3.annotate(model.replace('_', ' ').title(),
                    (training_times[i], accuracies[i]),
                    xytext=(5, 5), textcoords='offset points', fontsize=8)

    ax3.set_xlabel('Training Time (minutes)')
    ax3.set_ylabel('Test Accuracy (%)')
    ax3.set_title('Training Efficiency\n(Bubble size = efficiency)', fontweight='bold', fontsize=14)
    ax3.grid(True, alpha=0.3)

    # 4. Training Curves (Bottom Left - spans 2 columns)
    ax4 = plt.subplot(2, 3, (4, 5))

    for model_name, result in valid_results.items():
        if 'train_history' in result and 'test_history' in result:
            train_hist = result['train_history']
            test_hist = result['test_history']
            epochs = range(1, len(train_hist['accuracy']) + 1)

            # Plot only test accuracy to avoid clutter
            ax4.plot(epochs, [x*100 for x in test_hist['accuracy']],
                    label=f'{model_name.replace("_", " ").title()}',
                    linewidth=2, alpha=0.8)

    ax4.set_xlabel('Epoch')
    ax4.set_ylabel('Validation Accuracy (%)')
    ax4.set_title('Training Convergence Curves', fontweight='bold', fontsize=14)
    ax4.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax4.grid(True, alpha=0.3)

    # 5. Model Categories Comparison (Bottom Right)
    ax5 = plt.subplot(2, 3, 6)

    # Categorize models
    categories = {
        'Baseline': ['simple_cnn', 'lenet', 'alexnet'],
        'Modern CNN': ['resnet18', 'resnet50', 'efficientnet_b0', 'efficientnet_b2', 'regnet_y_400mf'],
        'Transformer': ['vit_base_16', 'deit_small', 'swin_tiny'],
        'Mobile': ['mobilenet_v3_small', 'efficientnet_lite0']
    }

    category_performance = {}
    for category, model_list in categories.items():
        category_accs = []
        for model in model_list:
            if model in valid_results:
                category_accs.append(valid_results[model]['final_test_acc']*100)

        if category_accs:
            category_performance[category] = {
                'mean': np.mean(category_accs),
                'std': np.std(category_accs),
                'count': len(category_accs)
            }

    if category_performance:
        cats = list(category_performance.keys())
        means = [category_performance[cat]['mean'] for cat in cats]
        stds = [category_performance[cat]['std'] for cat in cats]
        counts = [category_performance[cat]['count'] for cat in cats]

        bars = ax5.bar(cats, means, yerr=stds, capsize=5, alpha=0.7,
                      color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])

        # Add count labels
        for bar, count in zip(bars, counts):
            ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(stds)*0.1,
                    f'n={count}', ha='center', fontweight='bold')

        ax5.set_ylabel('Mean Accuracy (%)')
        ax5.set_title('Performance by Architecture Category', fontweight='bold', fontsize=14)
        ax5.tick_params(axis='x', rotation=45)
        ax5.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.savefig('complete_model_comparison.png', dpi=300, bbox_inches='tight')

    # save to drive as well
    try:
        drive_path = f"{DRIVE_BASE_PATH}/plots/complete_model_comparison.png"
        plt.savefig(drive_path, dpi=300, bbox_inches='tight')
        print(f"📈 Plot saved to Drive: {drive_path}")
    except Exception as e:
        print(f"⚠️ Could not save plot to Drive: {e}")
    plt.show()

def create_detailed_summary(results_dict):
    """Create detailed summary table and statistics"""

    # Filter valid results
    valid_results = {k: v for k, v in results_dict.items() if v is not None}

    summary_data = []
    for model_name, result in valid_results.items():

        # Get additional metrics if available
        precision = result.get('precision', 0)
        recall = result.get('recall', 0)
        f1_score = result.get('f1_score', 0)

        summary_data.append({
            'Model': model_name.replace('_', ' ').title(),
            'Test Accuracy (%)': f"{result['final_test_acc']*100:.2f}",
            'Training Time (min)': f"{result['training_time']/60:.1f}",
            'Efficiency (Acc%/min)': f"{(result['final_test_acc']*100)/(result['training_time']/60):.2f}",
            'Precision': f"{precision:.3f}" if precision > 0 else "N/A",
            'Recall': f"{recall:.3f}" if recall > 0 else "N/A",
            'F1-Score': f"{f1_score:.3f}" if f1_score > 0 else "N/A"
        })

    df = pd.DataFrame(summary_data)
    df = df.sort_values('Test Accuracy (%)', ascending=False)

    print("\n📊 COMPREHENSIVE RESULTS SUMMARY:")
    print("=" * 100)
    print(df.to_string(index=False))
    print("=" * 100)

    # Statistics
    accuracies = [float(row['Test Accuracy (%)']) for _, row in df.iterrows()]
    times = [float(row['Training Time (min)']) for _, row in df.iterrows()]

    print(f"\n📈 STATISTICS:")
    print(f"Models Tested: {len(valid_results)}")
    print(f"Best Accuracy: {max(accuracies):.2f}%")
    print(f"Worst Accuracy: {min(accuracies):.2f}%")
    print(f"Mean Accuracy: {np.mean(accuracies):.2f}% ± {np.std(accuracies):.2f}%")
    print(f"Total Training Time: {sum(times):.1f} minutes ({sum(times)/60:.1f} hours)")
    print(f"Average Training Time: {np.mean(times):.1f} minutes")

    return df


### Save Experiment Progress

In [18]:
def save_experiment_progress(results, current_batch, completed_batches):
    """Save experiment progress to Google Drive"""
    # Ensure Drive directories exist
    setup_drive_directories()

    progress_data = {
        'results': results,
        'current_batch': current_batch,
        'completed_batches': completed_batches,
        'timestamp': time.time()
    }

    # Save to Drive
    progress_path = f"{DRIVE_BASE_PATH}/progress/experiment_progress.json"
    with open(progress_path, 'w') as f:
        json.dump(progress_data, f, indent=2, default=str)

    # Also save backup with timestamp
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    backup_path = f"{DRIVE_BASE_PATH}/progress/progress_backup_{timestamp}.json"
    with open(backup_path, 'w') as f:
        json.dump(progress_data, f, indent=2, default=str)

    print(f"💾 Progress saved to Drive - Batch {current_batch} completed")
    print(f"📍 Location: {progress_path}")

### Load Experiment Progress

In [19]:
def load_experiment_progress():
    """Load previous experiment progress from Google Drive"""
    try:
        progress_path = f"{DRIVE_BASE_PATH}/progress/experiment_progress.json"
        if Path(progress_path).exists():
            with open(progress_path, 'r') as f:
                progress_data = json.load(f)
            print(f"📂 Loaded previous progress from Drive - {len(progress_data.get('completed_batches', []))} batches completed")
            return progress_data
        else:
            print("ℹ️ No previous progress found in Drive - starting fresh")
            return {'results': {}, 'current_batch': 0, 'completed_batches': []}
    except Exception as e:
        print(f"⚠️ Could not load progress from Drive: {e}")
        return {'results': {}, 'current_batch': 0, 'completed_batches': []}

### Estimated Time Running

In [20]:
def estimate_time_remaining(completed_batches, total_batches, start_time):
    """Estimate remaining time based on completed batches"""
    if len(completed_batches) == 0:
        return "Unknown"

    elapsed_time = time.time() - start_time
    avg_time_per_batch = elapsed_time / len(completed_batches)
    remaining_batches = total_batches - len(completed_batches)
    estimated_remaining = avg_time_per_batch * remaining_batches

    return f"{estimated_remaining/3600:.1f} hours"

# Main - Batch Execution

In [21]:
def run_complete_colab_experiment(batch_to_run=None, max_time_hours=10):
    """
    Complete experiment function with BATCH execution to avoid Colab timeouts

    Args:
        batch_to_run: Specific batch number to run (0-5), or None to run next batch
        max_time_hours: Maximum time to run before stopping (default 10 hours)
    """
    print("🚀 COMPLETE CIFAR-100 Architecture Comparison - BATCH EXECUTION MODE")
    print("🎯 Designed to work within Colab's 12-hour limit")
    print("=" * 80)

    # Load previous progress
    progress = load_experiment_progress()
    results = progress.get('results', {})
    completed_batches = progress.get('completed_batches', [])

    # Check GPU availability
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"🔥 Device: {device}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name()}")
        print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
        print(f"Mixed Precision: Enabled")
    else:
        print("⚠️ Running on CPU - will be slower")

    # Load data once
    print(f"\n📊 Loading CIFAR-100 dataset...")
    train_loader, test_loader, class_names = get_cifar100_dataloaders(
        batch_size=64 if torch.cuda.is_available() else 32,
        num_workers=2
    )

    # Define model batches for manageable execution
    model_batches = {
        0: {'name': 'Logistic Regression', 'models': ['logistic_regression']},
        1: {'name': 'Baseline Models', 'models': ['simple_cnn', 'lenet', 'alexnet']},
        2: {'name': 'Efficient CNNs', 'models': ['resnet18', 'efficientnet_b0', 'mobilenet_v3_small']},
        3: {'name': 'Powerful CNNs', 'models': ['resnet50', 'efficientnet_b2', 'regnet_y_400mf']},
        4: {'name': 'Vision Transformers', 'models': ['vit_base_16', 'deit_small', 'swin_tiny']},
        5: {'name': 'Mobile Optimized', 'models': ['efficientnet_lite0']},
        6: {'name': 'Swin Tiny Retry', 'models': ['swin_tiny']}
    }

    total_models = sum(len(batch['models']) for batch in model_batches.values())
    print(f"🎯 Total models across all batches: {total_models}")
    print(f"📦 Organized into {len(model_batches)} batches")

    # Determine which batch to run
    if batch_to_run is not None:
        if batch_to_run not in model_batches:
            print(f"❌ Invalid batch number: {batch_to_run}. Valid batches: 0-{len(model_batches)-1}")
            return results, None

        if batch_to_run in completed_batches:
            print(f"⚠️ Batch {batch_to_run} already completed. Use a different batch number.")
            return results, None

        batches_to_run = [batch_to_run]
        print(f"🎯 Running specific batch: {batch_to_run} ({model_batches[batch_to_run]['name']})")
    else:
        # Find next uncompleted batch
        batches_to_run = [b for b in model_batches.keys() if b not in completed_batches]
        if not batches_to_run:
            print("✅ All batches completed! Generating final results...")
            return generate_final_results(results)

        print(f"🎯 Running next batch: {batches_to_run[0]} ({model_batches[batches_to_run[0]]['name']})")
        batches_to_run = [batches_to_run[0]]  # Run only the next batch

    # Time tracking
    experiment_start_time = time.time()
    max_time_seconds = max_time_hours * 3600

    # Execute selected batch
    for batch_num in batches_to_run:
        batch_info = model_batches[batch_num]
        batch_start_time = time.time()

        print(f"\n🏗️ BATCH {batch_num}: {batch_info['name'].upper()}")
        print("=" * 70)
        print(f"📋 Models in this batch: {', '.join(batch_info['models'])}")

        batch_results = {}

        for model_idx, model_name in enumerate(batch_info['models']):
            # Check time limit
            elapsed_time = time.time() - experiment_start_time
            if elapsed_time > max_time_seconds:
                print(f"⏰ Time limit reached ({max_time_hours} hours). Stopping execution.")
                print(f"💾 Progress saved. Resume with batch {batch_num} to continue.")
                return results, None

            print(f"\n🔧 Training {model_name} ({model_idx+1}/{len(batch_info['models'])})")
            print("-" * 50)

            try:
                if model_name == 'logistic_regression':
                    # Special handling for logistic regression
                    result = train_logistic_regression_baseline(train_loader, test_loader)
                    batch_results[model_name] = result
                    print(f"✅ {model_name}: {result['final_test_acc']*100:.2f}%")

                else:
                    # Neural network models
                    config = get_training_config(model_name, str(device))
                    print(f"⚙️ Config: LR={config['learning_rate']}, BS={config['batch_size']}, Epochs={config['epochs']}")

                    # Create model
                    model = get_model(model_name, num_classes=100, pretrained=True)

                    # Count parameters
                    total_params = sum(p.numel() for p in model.parameters())
                    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
                    print(f"📊 Model size: {total_params:,} total parameters, {trainable_params:,} trainable")

                    # Create custom data loaders if needed
                    if config['batch_size'] != 64:
                        model_train_loader, model_test_loader, _ = get_cifar100_dataloaders(
                            batch_size=config['batch_size'], num_workers=2
                        )
                    else:
                        model_train_loader, model_test_loader = train_loader, test_loader

                    # Create trainer
                    trainer = ColabTrainer(
                        model,
                        device=config['device'],
                        learning_rate=config['learning_rate'],
                        weight_decay=config['weight_decay'],
                        model_name=model_name
                    )

                    # Train model
                    result = trainer.train(
                        model_train_loader,
                        model_test_loader,
                        epochs=config['epochs'],
                        early_stopping_patience=config['early_stopping_patience']
                    )

                    result['model_name'] = model_name
                    result['total_parameters'] = total_params
                    result['trainable_parameters'] = trainable_params
                    batch_results[model_name] = result

                    print(f"✅ {model_name} completed successfully!")
                    print(f"📊 Final Test Accuracy: {result['final_test_acc']*100:.2f}%")
                    print(f"⏱️ Training Time: {result['training_time']/60:.1f} minutes")

                    # Clean up GPU memory
                    del model, trainer
                    gc.collect()
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                        torch.cuda.synchronize()

                # Show remaining time estimate
                elapsed_batch_time = time.time() - batch_start_time
                remaining_models = len(batch_info['models']) - (model_idx + 1)
                if remaining_models > 0:
                    avg_time_per_model = elapsed_batch_time / (model_idx + 1)
                    est_remaining = avg_time_per_model * remaining_models
                    print(f"⏳ Estimated time for remaining models in batch: {est_remaining/60:.1f} minutes")

            except RuntimeError as e:
                if "CUDA out of memory" in str(e):
                    print(f"🔴 CUDA OOM for {model_name} - Try reducing batch size")
                else:
                    print(f"🔴 Runtime error for {model_name}: {e}")
                batch_results[model_name] = None

            except Exception as e:
                print(f"🔴 Unexpected error for {model_name}: {e}")
                batch_results[model_name] = None

            finally:
                # Always clean up
                #if torch.cuda.is_available():
                #    torch.cuda.empty_cache()
                try:
                    if 'model' in locals():
                        del model
                    if 'trainer' in locals():
                        del trainer
                except:
                    pass
                gc.collect()  # ←← ADD THIS LINE
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    torch.cuda.synchronize()

        # Update results and save progress
        results.update(batch_results)
        completed_batches.append(batch_num)
        save_experiment_progress(results, batch_num, completed_batches)

        batch_time = time.time() - batch_start_time
        successful_in_batch = sum(1 for v in batch_results.values() if v is not None)

        print(f"\n📊 BATCH {batch_num} SUMMARY:")
        print(f"✅ Successfully trained: {successful_in_batch}/{len(batch_info['models'])} models")
        print(f"⏱️ Batch time: {batch_time/60:.1f} minutes")

        # Show best model in this batch
        if successful_in_batch > 0:
            valid_batch_results = {k: v for k, v in batch_results.items() if v is not None}
            best_in_batch = max(valid_batch_results.keys(), key=lambda x: valid_batch_results[x]['final_test_acc'])
            best_acc = valid_batch_results[best_in_batch]['final_test_acc'] * 100
            print(f"🏆 Best in batch: {best_in_batch} ({best_acc:.2f}%)")

    # Generate intermediate results if we have any
    successful_models = sum(1 for v in results.values() if v is not None)
    if successful_models > 0:
        print(f"\n📈 Generating Intermediate Results...")
        print(f"✅ Total models completed so far: {successful_models}")

        # Generate summary and visualization with current results
        df_summary = create_detailed_summary(results)
        plot_comprehensive_results(results)

        # Save intermediate results
        json_results = {}
        for model, result in results.items():
            if result is not None:
                json_results[model] = {
                    'final_test_acc': float(result['final_test_acc']),
                    'training_time': float(result['training_time']),
                    'model_name': result.get('model_name', model),
                    'total_parameters': result.get('total_parameters', 0),
                    'trainable_parameters': result.get('trainable_parameters', 0)
                }

        # Save intermediate results to Drive
        results_path = f"{DRIVE_BASE_PATH}/results/intermediate_results_batch_{batch_num}.json"
        with open(results_path, 'w') as f:
            json.dump(json_results, f, indent=2)

        # Also save plots to Drive
        plot_path = f"{DRIVE_BASE_PATH}/plots/intermediate_results_batch_{batch_num}.png"
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')

        print(f"💾 Intermediate results saved to Drive:")
        print(f"   📊 Data: {results_path}")
        print(f"   📈 Plot: {plot_path}")

    # Check if all batches are complete
    remaining_batches = [b for b in model_batches.keys() if b not in completed_batches]

    if not remaining_batches:
        print(f"\n🎉 ALL BATCHES COMPLETED!")
        return generate_final_results(results)
    else:
        print(f"\n⏭️ NEXT STEPS:")
        print(f"📋 Completed batches: {sorted(completed_batches)}")
        print(f"📋 Remaining batches: {sorted(remaining_batches)}")
        print(f"🔄 To continue, run: run_complete_colab_experiment(batch_to_run={remaining_batches[0]})")
        print(f"🔄 Or run: run_complete_colab_experiment() to auto-continue")


        #return results, df_summary if successful_models > 0 else None                                           #-> Prints the complete results.json
        print("🎉 Experiment completed successfully!")
        print(f"📂 All results saved to: {DRIVE_BASE_PATH}")
        return df_summary

### Generate Final Results

In [22]:
def generate_final_results(results):
    """Generate final comprehensive results when all batches are complete"""
    print(f"\n🎊 EXPERIMENT COMPLETE - GENERATING FINAL RESULTS!")
    print("=" * 70)

    successful_models = sum(1 for v in results.values() if v is not None)
    failed_models = sum(1 for v in results.values() if v is None)
    total_models = len(results)

    print(f"✅ Successfully trained: {successful_models}/{total_models} models")
    print(f"❌ Failed models: {failed_models}")

    if successful_models > 0:
        # Create detailed summary
        df_summary = create_detailed_summary(results)

        # Create comprehensive visualizations
        print(f"\n🎨 Creating Final Comprehensive Visualizations...")
        plot_comprehensive_results(results)

        # Prepare final results data - ADD THIS PART:
        json_results = {}
        for model, result in results.items():
            if result is not None:
                json_results[model] = {
                    'final_test_acc': float(result['final_test_acc']),
                    'training_time': float(result['training_time']),
                    'model_name': result.get('model_name', model),
                    'total_parameters': result.get('total_parameters', 0),
                    'trainable_parameters': result.get('trainable_parameters', 0)
                }

        # Save final results to Drive
        print(f"\n💾 Saving Final Results to Google Drive...")

        final_results_path = f"{DRIVE_BASE_PATH}/results/FINAL_experiment_results.json"
        with open(final_results_path, 'w') as f:
            json.dump(json_results, f, indent=2)

        final_csv_path = f"{DRIVE_BASE_PATH}/results/FINAL_results_summary.csv"
        df_summary.to_csv(final_csv_path, index=False)

        final_plot_path = f"{DRIVE_BASE_PATH}/plots/FINAL_model_comparison.png"
        plt.savefig(final_plot_path, dpi=300, bbox_inches='tight')

        print("🎯 FINAL RESULTS saved to Google Drive:")
        print(f"   📊 Data: {final_results_path}")
        print(f"   📋 Summary: {final_csv_path}")
        print(f"   📈 Plot: {final_plot_path}")

        # Clean up progress file from Drive
        progress_path = f"{DRIVE_BASE_PATH}/progress/experiment_progress.json"
        if Path(progress_path).exists():
            os.remove(progress_path)
            print("🧹 Progress tracking file cleaned up from Drive")

        return results, df_summary
    else:
        print("❌ No models completed successfully")
        return results, None

### Helper Functions - Bacth Execution

In [23]:
def show_batch_status():
    """Show current experiment status"""
    progress = load_experiment_progress()
    completed_batches = progress.get('completed_batches', [])

    model_batches = {
        0: {'name': 'Logistic Regression', 'models': ['logistic_regression']},
        1: {'name': 'Baseline Models', 'models': ['simple_cnn', 'lenet', 'alexnet']},
        2: {'name': 'Efficient CNNs', 'models': ['resnet18', 'efficientnet_b0', 'mobilenet_v3_small']},
        3: {'name': 'Powerful CNNs', 'models': ['resnet50', 'efficientnet_b2', 'regnet_y_400mf']},
        4: {'name': 'Vision Transformers', 'models': ['vit_base_16', 'deit_small', 'swin_tiny']},
        5: {'name': 'Mobile Optimized', 'models': ['efficientnet_lite0']},
        6: {'name': 'Swin Tiny Retry', 'models': ['swin_tiny']}
    }

    print("📊 EXPERIMENT STATUS:")
    print("=" * 50)

    for batch_num, batch_info in model_batches.items():
        status = "✅ COMPLETED" if batch_num in completed_batches else "⏳ PENDING"
        print(f"Batch {batch_num}: {batch_info['name']} - {status}")
        print(f"   Models: {', '.join(batch_info['models'])}")

    remaining = [b for b in model_batches.keys() if b not in completed_batches]
    if remaining:
        print(f"\n⏭️ Next batch to run: {remaining[0]} ({model_batches[remaining[0]]['name']})")
        print(f"🔄 Run: run_complete_colab_experiment(batch_to_run={remaining[0]})")
    else:
        print(f"\n🎉 All batches completed!")

def reset_experiment():
    """Reset experiment progress from Google Drive (use with caution!)"""
    progress_path = f"{DRIVE_BASE_PATH}/progress/experiment_progress.json"
    if Path(progress_path).exists():
        os.remove(progress_path)
        print("🧹 Experiment progress reset in Drive. Starting fresh!")
    else:
        print("ℹ️ No progress file found in Drive. Already starting fresh!")

def backup_drive_results():
    """Create a timestamped backup of all results in Drive"""
    setup_drive_directories()
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    backup_dir = f"{DRIVE_BASE_PATH}/backups/backup_{timestamp}"
    os.makedirs(backup_dir, exist_ok=True)

    # Copy all results to backup
    import shutil
    source_dirs = ['results', 'models', 'plots', 'progress']

    for source_dir in source_dirs:
        source_path = f"{DRIVE_BASE_PATH}/{source_dir}"
        if os.path.exists(source_path):
            dest_path = f"{backup_dir}/{source_dir}"
            shutil.copytree(source_path, dest_path, dirs_exist_ok=True)

    print(f"💾 Backup created at: {backup_dir}")

### RESET Specific Batch

To reset any specific batch follow these steps:
1. Manually remove `{modelname}_checpoint.pth` and `{modelname}_best.pth` file from `your-directory/CIFAR100_Experiment/models`
2. Run the below function `reset_specific_batch(batch_number={#})`
3. `run_complete_colab_experiment(batch_to_run={#})` again.

In [24]:
def reset_specific_batch(batch_number):
    """Reset progress for a specific batch"""
    progress = load_experiment_progress()
    if batch_number in progress['completed_batches']:
        progress['completed_batches'].remove(batch_number)
        save_experiment_progress(progress['results'], progress['current_batch'], progress['completed_batches'])
        print(f"✅ Reset batch {batch_number} - can now re-run")
    else:
        print(f"ℹ️ Batch {batch_number} was not marked as completed")

# Run this to reset batch 3:
#reset_specific_batch(3)

### Execute - Batch Mode

In [ ]:
if __name__ == "__main__":
    print("🎬 Starting CIFAR-100 Architecture Comparison - BATCH MODE")
    print("📋 This will train models in manageable batches!")
    print("⏱️ Each batch takes 1-3 hours, fits in Colab's limits")
    print("🔄 Automatically resumes from where you left off")
    print("🚀 Let's begin!\n")

    # Show current status first
    show_batch_status()
    print()

    # Run the next batch
    results, summary = run_complete_colab_experiment()

🎬 Starting CIFAR-100 Architecture Comparison - BATCH MODE
📋 This will train models in manageable batches!
⏱️ Each batch takes 1-3 hours, fits in Colab's limits
🔄 Automatically resumes from where you left off
🚀 Let's begin!

ℹ️ No previous progress found in Drive - starting fresh
📊 EXPERIMENT STATUS:
Batch 0: Logistic Regression - ⏳ PENDING
   Models: logistic_regression
Batch 1: Baseline Models - ⏳ PENDING
   Models: simple_cnn, lenet, alexnet
Batch 2: Efficient CNNs - ⏳ PENDING
   Models: resnet18, efficientnet_b0, mobilenet_v3_small
Batch 3: Powerful CNNs - ⏳ PENDING
   Models: resnet50, efficientnet_b2, regnet_y_400mf
Batch 4: Vision Transformers - ⏳ PENDING
   Models: vit_base_16, deit_small, swin_tiny
Batch 5: Mobile Optimized - ⏳ PENDING
   Models: efficientnet_lite0

⏭️ Next batch to run: 0 (Logistic Regression)
🔄 Run: run_complete_colab_experiment(batch_to_run=0)

🚀 COMPLETE CIFAR-100 Architecture Comparison - BATCH EXECUTION MODE
🎯 Designed to work within Colab's 12-hour limit

100%|██████████| 169M/169M [00:12<00:00, 13.1MB/s]


✅ CIFAR-100 loaded successfully!
   Training samples: 50,000
   Test samples: 10,000
   Classes: 100
🎯 Total models across all batches: 14
📦 Organized into 6 batches
🎯 Running next batch: 0 (Logistic Regression)

🏗️ BATCH 0: LOGISTIC REGRESSION
📋 Models in this batch: logistic_regression

🔧 Training logistic_regression (1/1)
--------------------------------------------------
📊 Training Logistic Regression Baseline...
Converting training data...


Loading train data: 100%|██████████| 50/50 [00:12<00:00,  4.14it/s]


Converting test data...


Loading test data: 100%|██████████| 10/10 [00:02<00:00,  4.21it/s]


Training data shape: (50000, 3072)
Test data shape: (10000, 3072)
Normalizing features...
Fitting logistic regression model...
Making predictions...
✅ Logistic Regression Results:
   Training Accuracy: 62.11%
   Test Accuracy: 11.90%
   Training Time: 3677.20s
✅ logistic_regression: 11.90%
📁 Drive directories created at: /content/drive/MyDrive/CIFAR100_Experiment
💾 Progress saved to Drive - Batch 0 completed
📍 Location: /content/drive/MyDrive/CIFAR100_Experiment/progress/experiment_progress.json

📊 BATCH 0 SUMMARY:
✅ Successfully trained: 1/1 models
⏱️ Batch time: 61.6 minutes
🏆 Best in batch: logistic_regression (11.90%)

📈 Generating Intermediate Results...
✅ Total models completed so far: 1

📊 COMPREHENSIVE RESULTS SUMMARY:
              Model Test Accuracy (%) Training Time (min) Efficiency (Acc%/min) Precision Recall F1-Score
Logistic Regression             11.90                61.3                  0.19       N/A    N/A      N/A

📈 STATISTICS:
Models Tested: 1
Best Accuracy: 11.9

In [ ]:
run_complete_colab_experiment(batch_to_run=1)

🚀 COMPLETE CIFAR-100 Architecture Comparison - BATCH EXECUTION MODE
🎯 Designed to work within Colab's 12-hour limit
📂 Loaded previous progress from Drive - 1 batches completed
🔥 Device: cuda
GPU: NVIDIA L4
Memory: 23.8 GB
Mixed Precision: Enabled

📊 Loading CIFAR-100 dataset...
✅ CIFAR-100 loaded successfully!
   Training samples: 50,000
   Test samples: 10,000
   Classes: 100
🎯 Total models across all batches: 14
📦 Organized into 6 batches
🎯 Running specific batch: 1 (Baseline Models)

🏗️ BATCH 1: BASELINE MODELS
📋 Models in this batch: simple_cnn, lenet, alexnet

🔧 Training simple_cnn (1/3)
--------------------------------------------------
⚙️ Config: LR=0.002, BS=64, Epochs=25
📊 Model size: 3,356,324 total parameters, 3,356,324 trainable
📁 Drive directories created at: /content/drive/MyDrive/CIFAR100_Experiment
🚀 Training simple_cnn on cuda
Mixed Precision: True
💾 Checkpoints will be saved to: /content/drive/MyDrive/CIFAR100_Experiment/models


Evaluating: 100%|██████████| 157/157 [00:08<00:00, 18.12it/s]


Epoch [1/25]
Train Loss: 4.1390, Train Acc: 0.0817
Test Loss: 3.8108, Test Acc: 0.1463
Best Acc: 0.1463, Patience: 0/5
💾 Best model saved: /content/drive/MyDrive/CIFAR100_Experiment/models/simple_cnn_best.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/simple_cnn_checkpoint.pth
------------------------------------------------------------


Evaluating: 100%|██████████| 157/157 [00:08<00:00, 18.25it/s]


Epoch [2/25]
Train Loss: 3.8646, Train Acc: 0.1387
Test Loss: 3.6016, Test Acc: 0.2019
Best Acc: 0.2019, Patience: 0/5
💾 Best model saved: /content/drive/MyDrive/CIFAR100_Experiment/models/simple_cnn_best.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/simple_cnn_checkpoint.pth
------------------------------------------------------------


...

Evaluating: 100%|██████████| 313/313 [00:08<00:00, 35.62it/s]


Epoch [19/20]
Train Loss: 3.0858, Train Acc: 0.3179
Test Loss: 3.0642, Test Acc: 0.3374
Best Acc: 0.3380, Patience: 1/5
------------------------------------------------------------


Evaluating: 100%|██████████| 313/313 [00:09<00:00, 31.47it/s]


Epoch [20/20]
Train Loss: 3.0749, Train Acc: 0.3198
Test Loss: 3.0611, Test Acc: 0.3381
Best Acc: 0.3381, Patience: 0/5
💾 Best model saved: /content/drive/MyDrive/CIFAR100_Experiment/models/alexnet_best.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/alexnet_checkpoint.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/alexnet_checkpoint.pth
------------------------------------------------------------
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/alexnet_checkpoint.pth
✅ alexnet completed successfully!
📊 Final Test Accuracy: 33.81%
⏱️ Training Time: 64.0 minutes
📁 Drive directories created at: /content/drive/MyDrive/CIFAR100_Experiment
💾 Progress saved to Drive - Batch 1 completed
📍 Location: /content/drive/MyDrive/CIFAR100_Experiment/progress/experiment_progress.json

📊 BATCH 1 SUMMARY:
✅ Successfully trained: 3/3 models
⏱️ Batch time: 215.9 minutes
🏆 Best in batch: simple_cnn (38.95%)

📈 Generating Intermediate 

({'logistic_regression': {'final_test_acc': 0.119,
   'training_time': 3677.2010815143585,
   'model_name': 'Logistic Regression'},
  'simple_cnn': {'final_test_acc': 0.3895,
   'training_time': 4470.086376667023,
   'train_history': {'loss': [4.139035476747986,
     3.8645806836959955,
     3.7233065653335102,
     3.6224190234528173,
     3.5421145538539838,
     3.457422825381579,
     3.39347879691502,
     3.3457077022098822,
     3.303953140288058,
     3.261827409114984,
     3.238922540184177,
     3.200535819963421,
     3.1789978440765223,
     3.152563854556559,
     3.1295263974562935,
     3.1101587471144887,
     3.093657934147379,
     3.089950628597718,
     3.0738695698321017,
     3.0726713963481775,
     3.073311245349972,
     3.0718019786088364,
     3.0677469548057106,
     3.076273701075093],
    'accuracy': [0.08172,
     0.13874,
     0.16846,
     0.19298,
     0.21298,
     0.23084,
     0.24666,
     0.26006,
     0.2717,
     0.27984,
     0.28866,
     0.2

In [ ]:
run_complete_colab_experiment(batch_to_run=2)

🚀 COMPLETE CIFAR-100 Architecture Comparison - BATCH EXECUTION MODE
🎯 Designed to work within Colab's 12-hour limit
📂 Loaded previous progress from Drive - 2 batches completed
🔥 Device: cuda
GPU: NVIDIA L4
Memory: 23.8 GB
Mixed Precision: Enabled

📊 Loading CIFAR-100 dataset...
✅ CIFAR-100 loaded successfully!
   Training samples: 50,000
   Test samples: 10,000
   Classes: 100
🎯 Total models across all batches: 14
📦 Organized into 6 batches
🎯 Running specific batch: 2 (Efficient CNNs)

🏗️ BATCH 2: EFFICIENT CNNS
📋 Models in this batch: resnet18, efficientnet_b0, mobilenet_v3_small

🔧 Training resnet18 (1/3)
--------------------------------------------------
⚙️ Config: LR=0.001, BS=64, Epochs=15


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 226MB/s]


📊 Model size: 11,227,812 total parameters, 11,227,812 trainable
📁 Drive directories created at: /content/drive/MyDrive/CIFAR100_Experiment
🚀 Training resnet18 on cuda
Mixed Precision: True
💾 Checkpoints will be saved to: /content/drive/MyDrive/CIFAR100_Experiment/models


Evaluating: 100%|██████████| 157/157 [00:08<00:00, 17.87it/s]


Epoch [1/15]
Train Loss: 2.6597, Train Acc: 0.4283
Test Loss: 2.3617, Test Acc: 0.5144
Best Acc: 0.5144, Patience: 0/5
💾 Best model saved: /content/drive/MyDrive/CIFAR100_Experiment/models/resnet18_best.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/resnet18_checkpoint.pth
------------------------------------------------------------


...

Evaluating: 100%|██████████| 105/105 [00:08<00:00, 12.17it/s]


Epoch [14/15]
Train Loss: 0.9173, Train Acc: 0.9853
Test Loss: inf, Test Acc: 0.7918
Best Acc: 0.7918, Patience: 0/5
💾 Best model saved: /content/drive/MyDrive/CIFAR100_Experiment/models/mobilenet_v3_small_best.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/mobilenet_v3_small_checkpoint.pth
------------------------------------------------------------


Evaluating: 100%|██████████| 105/105 [00:08<00:00, 12.23it/s]


Epoch [15/15]
Train Loss: 0.9008, Train Acc: 0.9897
Test Loss: inf, Test Acc: 0.7935
Best Acc: 0.7935, Patience: 0/5
💾 Best model saved: /content/drive/MyDrive/CIFAR100_Experiment/models/mobilenet_v3_small_best.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/mobilenet_v3_small_checkpoint.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/mobilenet_v3_small_checkpoint.pth
------------------------------------------------------------
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/mobilenet_v3_small_checkpoint.pth
✅ mobilenet_v3_small completed successfully!
📊 Final Test Accuracy: 79.35%
⏱️ Training Time: 46.6 minutes
📁 Drive directories created at: /content/drive/MyDrive/CIFAR100_Experiment
💾 Progress saved to Drive - Batch 2 completed
📍 Location: /content/drive/MyDrive/CIFAR100_Experiment/progress/experiment_progress.json

📊 BATCH 2 SUMMARY:
✅ Successfully trained: 3/3 models
⏱️ Batch time: 141.2 minutes
🏆 Best in ba

,Model,Test Accuracy (%),Training Time (min),Efficiency (Acc%/min),Precision,Recall,F1-Score
5,Efficientnet B0,84.60,47.3,1.79,N/A,N/A,N/A
6,Mobilenet V3 Small,79.35,46.6,1.70,N/A,N/A,N/A
4,Resnet18,78.56,47.2,1.66,N/A,N/A,N/A
1,Simple Cnn,38.95,74.5,0.52,N/A,N/A,N/A
3,Alexnet,33.81,64.0,0.53,N/A,N/A,N/A
2,Lenet,30.16,77.4,0.39,N/A,N/A,N/A
0,Logistic Regression,11.90,61.3,0.19,N/A,N/A,N/A


In [ ]:
run_complete_colab_experiment(batch_to_run=3)

🚀 COMPLETE CIFAR-100 Architecture Comparison - BATCH EXECUTION MODE
🎯 Designed to work within Colab's 12-hour limit
📂 Loaded previous progress from Drive - 3 batches completed
🔥 Device: cuda
GPU: NVIDIA L4
Memory: 23.8 GB
Mixed Precision: Enabled

📊 Loading CIFAR-100 dataset...


100%|██████████| 169M/169M [00:04<00:00, 41.5MB/s]


✅ CIFAR-100 loaded successfully!
   Training samples: 50,000
   Test samples: 10,000
   Classes: 100
🎯 Total models across all batches: 14
📦 Organized into 6 batches
🎯 Running specific batch: 3 (Powerful CNNs)

🏗️ BATCH 3: POWERFUL CNNS
📋 Models in this batch: resnet50, efficientnet_b2, regnet_y_400mf

🔧 Training resnet50 (1/3)
--------------------------------------------------
⚙️ Config: LR=0.0008, BS=32, Epochs=20


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 177MB/s]


📊 Model size: 23,712,932 total parameters, 23,712,932 trainable
✅ CIFAR-100 loaded successfully!
   Training samples: 50,000
   Test samples: 10,000
   Classes: 100
📁 Drive directories created at: /content/drive/MyDrive/CIFAR100_Experiment
🚀 Training resnet50 on cuda
Mixed Precision: True
💾 Checkpoints will be saved to: /content/drive/MyDrive/CIFAR100_Experiment/models
📂 Loading checkpoint from Drive: /content/drive/MyDrive/CIFAR100_Experiment/models/resnet50_checkpoint.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/resnet50_checkpoint.pth
🔴 Unexpected error for resnet50: cannot access local variable 'predictions' where it is not associated with a value

🔧 Training efficientnet_b2 (2/3)
--------------------------------------------------
⚙️ Config: LR=0.0008, BS=32, Epochs=20


...

Evaluating: 100%|██████████| 157/157 [00:08<00:00, 18.44it/s]


Epoch [14/15]
Train Loss: 0.8716, Train Acc: 0.9791
Test Loss: 1.3779, Test Acc: 0.8221
Best Acc: 0.8221, Patience: 0/5
💾 Best model saved: /content/drive/MyDrive/CIFAR100_Experiment/models/regnet_y_400mf_best.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/regnet_y_400mf_checkpoint.pth
------------------------------------------------------------


Evaluating: 100%|██████████| 157/157 [00:08<00:00, 18.39it/s]


Epoch [15/15]
Train Loss: 0.8518, Train Acc: 0.9855
Test Loss: 1.3753, Test Acc: 0.8199
Best Acc: 0.8221, Patience: 1/5
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/regnet_y_400mf_checkpoint.pth
------------------------------------------------------------
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/regnet_y_400mf_checkpoint.pth
✅ regnet_y_400mf completed successfully!
📊 Final Test Accuracy: 82.21%
⏱️ Training Time: 15.6 minutes
📁 Drive directories created at: /content/drive/MyDrive/CIFAR100_Experiment
💾 Progress saved to Drive - Batch 3 completed
📍 Location: /content/drive/MyDrive/CIFAR100_Experiment/progress/experiment_progress.json

📊 BATCH 3 SUMMARY:
✅ Successfully trained: 1/3 models
⏱️ Batch time: 16.0 minutes
🏆 Best in batch: regnet_y_400mf (82.21%)

📈 Generating Intermediate Results...
✅ Total models completed so far: 8

📊 COMPREHENSIVE RESULTS SUMMARY:
              Model Test Accuracy (%) Training Time (min) Efficiency (Acc%/m

,Model,Test Accuracy (%),Training Time (min),Efficiency (Acc%/min),Precision,Recall,F1-Score
5,Efficientnet B0,84.60,47.3,1.79,N/A,N/A,N/A
7,Regnet Y 400Mf,82.21,15.6,5.28,N/A,N/A,N/A
6,Mobilenet V3 Small,79.35,46.6,1.70,N/A,N/A,N/A
4,Resnet18,78.56,47.2,1.66,N/A,N/A,N/A
1,Simple Cnn,38.95,74.5,0.52,N/A,N/A,N/A
3,Alexnet,33.81,64.0,0.53,N/A,N/A,N/A
2,Lenet,30.16,77.4,0.39,N/A,N/A,N/A
0,Logistic Regression,11.90,61.3,0.19,N/A,N/A,N/A


In [ ]:
run_complete_colab_experiment(batch_to_run=3)

🚀 COMPLETE CIFAR-100 Architecture Comparison - BATCH EXECUTION MODE
🎯 Designed to work within Colab's 12-hour limit
📂 Loaded previous progress from Drive - 3 batches completed
🔥 Device: cuda
GPU: NVIDIA L4
Memory: 23.8 GB
Mixed Precision: Enabled

📊 Loading CIFAR-100 dataset...
✅ CIFAR-100 loaded successfully!
   Training samples: 50,000
   Test samples: 10,000
   Classes: 100
🎯 Total models across all batches: 14
📦 Organized into 6 batches
🎯 Running specific batch: 3 (Powerful CNNs)

🏗️ BATCH 3: POWERFUL CNNS
📋 Models in this batch: resnet50, efficientnet_b2, regnet_y_400mf

🔧 Training resnet50 (1/3)
--------------------------------------------------
⚙️ Config: LR=0.0008, BS=32, Epochs=20
📊 Model size: 23,712,932 total parameters, 23,712,932 trainable
✅ CIFAR-100 loaded successfully!
   Training samples: 50,000
   Test samples: 10,000
   Classes: 100
📁 Drive directories created at: /content/drive/MyDrive/CIFAR100_Experiment
🚀 Training resnet50 on cuda
Mixed Precision: True
💾 Checkpoin

Evaluating: 100%|██████████| 313/313 [00:08<00:00, 35.01it/s]


Epoch [1/20]
Train Loss: 2.5037, Train Acc: 0.4746
Test Loss: 2.0201, Test Acc: 0.6172
Best Acc: 0.6172, Patience: 0/5
💾 Best model saved: /content/drive/MyDrive/CIFAR100_Experiment/models/resnet50_best.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/resnet50_checkpoint.pth
------------------------------------------------------------


Evaluating: 100%|██████████| 313/313 [00:08<00:00, 35.86it/s]


Epoch [2/20]
Train Loss: 1.8846, Train Acc: 0.6575
Test Loss: 1.7690, Test Acc: 0.6982
Best Acc: 0.6982, Patience: 0/5
💾 Best model saved: /content/drive/MyDrive/CIFAR100_Experiment/models/resnet50_best.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/resnet50_checkpoint.pth
------------------------------------------------------------


...

Evaluating: 100%|██████████| 157/157 [00:08<00:00, 18.24it/s]


Epoch [14/15]
Train Loss: 0.8695, Train Acc: 0.9796
Test Loss: 1.4012, Test Acc: 0.8135
Best Acc: 0.8141, Patience: 1/5
------------------------------------------------------------


Evaluating: 100%|██████████| 157/157 [00:08<00:00, 18.43it/s]


Epoch [15/15]
Train Loss: 0.8497, Train Acc: 0.9864
Test Loss: 1.3862, Test Acc: 0.8195
Best Acc: 0.8195, Patience: 0/5
💾 Best model saved: /content/drive/MyDrive/CIFAR100_Experiment/models/regnet_y_400mf_best.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/regnet_y_400mf_checkpoint.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/regnet_y_400mf_checkpoint.pth
------------------------------------------------------------
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/regnet_y_400mf_checkpoint.pth
✅ regnet_y_400mf completed successfully!
📊 Final Test Accuracy: 81.95%
⏱️ Training Time: 46.7 minutes
📁 Drive directories created at: /content/drive/MyDrive/CIFAR100_Experiment
💾 Progress saved to Drive - Batch 3 completed
📍 Location: /content/drive/MyDrive/CIFAR100_Experiment/progress/experiment_progress.json

📊 BATCH 3 SUMMARY:
✅ Successfully trained: 3/3 models
⏱️ Batch time: 172.6 minutes
🏆 Best in batch: efficientnet

,Model,Test Accuracy (%),Training Time (min),Efficiency (Acc%/min),Precision,Recall,F1-Score
8,Efficientnet B2,85.48,62.6,1.37,N/A,N/A,N/A
5,Efficientnet B0,84.60,47.3,1.79,N/A,N/A,N/A
7,Resnet50,82.90,63.2,1.31,N/A,N/A,N/A
9,Regnet Y 400Mf,81.95,46.7,1.76,N/A,N/A,N/A
6,Mobilenet V3 Small,79.35,46.6,1.70,N/A,N/A,N/A
4,Resnet18,78.56,47.2,1.66,N/A,N/A,N/A
1,Simple Cnn,38.95,74.5,0.52,N/A,N/A,N/A
3,Alexnet,33.81,64.0,0.53,N/A,N/A,N/A
2,Lenet,30.16,77.4,0.39,N/A,N/A,N/A
0,Logistic Regression,11.90,61.3,0.19,N/A,N/A,N/A


In [ ]:
run_complete_colab_experiment(batch_to_run=4)

🚀 COMPLETE CIFAR-100 Architecture Comparison - BATCH EXECUTION MODE
🎯 Designed to work within Colab's 12-hour limit
📂 Loaded previous progress from Drive - 4 batches completed
🔥 Device: cuda
GPU: NVIDIA L4
Memory: 23.8 GB
Mixed Precision: Enabled

📊 Loading CIFAR-100 dataset...
✅ CIFAR-100 loaded successfully!
   Training samples: 50,000
   Test samples: 10,000
   Classes: 100
🎯 Total models across all batches: 14
📦 Organized into 6 batches
🎯 Running specific batch: 4 (Vision Transformers)

🏗️ BATCH 4: VISION TRANSFORMERS
📋 Models in this batch: vit_base_16, deit_small, swin_tiny

🔧 Training vit_base_16 (1/3)
--------------------------------------------------
⚙️ Config: LR=0.0005, BS=24, Epochs=18


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

📊 Model size: 85,875,556 total parameters, 85,875,556 trainable
✅ CIFAR-100 loaded successfully!
   Training samples: 50,000
   Test samples: 10,000
   Classes: 100
📁 Drive directories created at: /content/drive/MyDrive/CIFAR100_Experiment
🚀 Training vit_base_16 on cuda
Mixed Precision: True
💾 Checkpoints will be saved to: /content/drive/MyDrive/CIFAR100_Experiment/models


Evaluating: 100%|██████████| 417/417 [00:12<00:00, 33.11it/s]


Epoch [1/18]
Train Loss: 4.1400, Train Acc: 0.0810
Test Loss: 3.9146, Test Acc: 0.1261
Best Acc: 0.1261, Patience: 0/7
💾 Best model saved: /content/drive/MyDrive/CIFAR100_Experiment/models/vit_base_16_best.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/vit_base_16_checkpoint.pth
------------------------------------------------------------


...

Evaluating: 100%|██████████| 417/417 [00:11<00:00, 36.89it/s]


Epoch [7/18]
Train Loss: 4.6105, Train Acc: 0.0093
Test Loss: 4.6086, Test Acc: 0.0100
Best Acc: 0.0303, Patience: 6/7
------------------------------------------------------------


Evaluating: 100%|██████████| 417/417 [00:11<00:00, 36.95it/s]


Epoch [8/18]
Train Loss: 4.6094, Train Acc: 0.0104
Test Loss: 4.6082, Test Acc: 0.0100
Best Acc: 0.0303, Patience: 7/7
------------------------------------------------------------
🛑 Early stopping triggered after 8 epochs
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/swin_tiny_checkpoint.pth
✅ swin_tiny completed successfully!
📊 Final Test Accuracy: 3.03%
⏱️ Training Time: 25.8 minutes
📁 Drive directories created at: /content/drive/MyDrive/CIFAR100_Experiment
💾 Progress saved to Drive - Batch 4 completed
📍 Location: /content/drive/MyDrive/CIFAR100_Experiment/progress/experiment_progress.json

📊 BATCH 4 SUMMARY:
✅ Successfully trained: 3/3 models
⏱️ Batch time: 161.2 minutes
🏆 Best in batch: deit_small (64.21%)

📈 Generating Intermediate Results...
✅ Total models completed so far: 13

📊 COMPREHENSIVE RESULTS SUMMARY:
              Model Test Accuracy (%) Training Time (min) Efficiency (Acc%/min) Precision Recall F1-Score
    Efficientnet B2             85.48     

,Model,Test Accuracy (%),Training Time (min),Efficiency (Acc%/min),Precision,Recall,F1-Score
8,Efficientnet B2,85.48,62.6,1.37,N/A,N/A,N/A
5,Efficientnet B0,84.60,47.3,1.79,N/A,N/A,N/A
7,Resnet50,82.90,63.2,1.31,N/A,N/A,N/A
9,Regnet Y 400Mf,81.95,46.7,1.76,N/A,N/A,N/A
6,Mobilenet V3 Small,79.35,46.6,1.70,N/A,N/A,N/A
4,Resnet18,78.56,47.2,1.66,N/A,N/A,N/A
11,Deit Small,64.21,57.3,1.12,N/A,N/A,N/A
10,Vit Base 16,54.79,77.8,0.70,N/A,N/A,N/A
1,Simple Cnn,38.95,74.5,0.52,N/A,N/A,N/A
3,Alexnet,33.81,64.0,0.53,N/A,N/A,N/A


In [ ]:
run_complete_colab_experiment(batch_to_run=5)

🚀 COMPLETE CIFAR-100 Architecture Comparison - BATCH EXECUTION MODE
🎯 Designed to work within Colab's 12-hour limit
📂 Loaded previous progress from Drive - 5 batches completed
🔥 Device: cuda
GPU: NVIDIA L4
Memory: 23.8 GB
Mixed Precision: Enabled

📊 Loading CIFAR-100 dataset...
✅ CIFAR-100 loaded successfully!
   Training samples: 50,000
   Test samples: 10,000
   Classes: 100
🎯 Total models across all batches: 14
📦 Organized into 6 batches
🎯 Running specific batch: 5 (Mobile Optimized)

🏗️ BATCH 5: MOBILE OPTIMIZED
📋 Models in this batch: efficientnet_lite0

🔧 Training efficientnet_lite0 (1/1)
--------------------------------------------------
⚙️ Config: LR=0.0012, BS=96, Epochs=15


model.safetensors:   0%|          | 0.00/18.8M [00:00<?, ?B/s]

📊 Model size: 3,499,108 total parameters, 3,499,108 trainable
✅ CIFAR-100 loaded successfully!
   Training samples: 50,000
   Test samples: 10,000
   Classes: 100
📁 Drive directories created at: /content/drive/MyDrive/CIFAR100_Experiment
🚀 Training efficientnet_lite0 on cuda
Mixed Precision: True
💾 Checkpoints will be saved to: /content/drive/MyDrive/CIFAR100_Experiment/models


Evaluating: 100%|██████████| 105/105 [00:08<00:00, 12.26it/s]


Epoch [1/15]
Train Loss: 2.1678, Train Acc: 0.5723
Test Loss: inf, Test Acc: 0.7080
Best Acc: 0.7080, Patience: 0/5
💾 Best model saved: /content/drive/MyDrive/CIFAR100_Experiment/models/efficientnet_lite0_best.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/efficientnet_lite0_checkpoint.pth
------------------------------------------------------------


...

Evaluating: 100%|██████████| 105/105 [00:08<00:00, 12.11it/s]


Epoch [14/15]
Train Loss: 0.8522, Train Acc: 0.9949
Test Loss: inf, Test Acc: 0.8314
Best Acc: 0.8314, Patience: 0/5
💾 Best model saved: /content/drive/MyDrive/CIFAR100_Experiment/models/efficientnet_lite0_best.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/efficientnet_lite0_checkpoint.pth
------------------------------------------------------------


Evaluating: 100%|██████████| 105/105 [00:08<00:00, 12.26it/s]


Epoch [15/15]
Train Loss: 0.8399, Train Acc: 0.9963
Test Loss: inf, Test Acc: 0.8309
Best Acc: 0.8314, Patience: 1/5
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/efficientnet_lite0_checkpoint.pth
------------------------------------------------------------
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/efficientnet_lite0_checkpoint.pth
✅ efficientnet_lite0 completed successfully!
📊 Final Test Accuracy: 83.14%
⏱️ Training Time: 46.2 minutes
📁 Drive directories created at: /content/drive/MyDrive/CIFAR100_Experiment
💾 Progress saved to Drive - Batch 5 completed
📍 Location: /content/drive/MyDrive/CIFAR100_Experiment/progress/experiment_progress.json

📊 BATCH 5 SUMMARY:
✅ Successfully trained: 1/1 models
⏱️ Batch time: 46.2 minutes
🏆 Best in batch: efficientnet_lite0 (83.14%)

📈 Generating Intermediate Results...
✅ Total models completed so far: 14

📊 COMPREHENSIVE RESULTS SUMMARY:
              Model Test Accuracy (%) Training Time (min) Effi

({'logistic_regression': {'final_test_acc': 0.119,
   'training_time': 3677.2010815143585,
   'model_name': 'Logistic Regression'},
  'simple_cnn': {'final_test_acc': 0.3895,
   'training_time': 4470.086376667023,
   'train_history': {'loss': [4.139035476747986,
     3.8645806836959955,
     3.7233065653335102,
     3.6224190234528173,
     3.5421145538539838,
     3.457422825381579,
     3.39347879691502,
     3.3457077022098822,
     3.303953140288058,
     3.261827409114984,
     3.238922540184177,
     3.200535819963421,
     3.1789978440765223,
     3.152563854556559,
     3.1295263974562935,
     3.1101587471144887,
     3.093657934147379,
     3.089950628597718,
     3.0738695698321017,
     3.0726713963481775,
     3.073311245349972,
     3.0718019786088364,
     3.0677469548057106,
     3.076273701075093],
    'accuracy': [0.08172,
     0.13874,
     0.16846,
     0.19298,
     0.21298,
     0.23084,
     0.24666,
     0.26006,
     0.2717,
     0.27984,
     0.28866,
     0.2

#### I couldn't wrap my head around SwinTiny performing so badly, so I had to run the model again to map out the results.

In [ ]:
run_complete_colab_experiment(batch_to_run=6)

🚀 COMPLETE CIFAR-100 Architecture Comparison - BATCH EXECUTION MODE
🎯 Designed to work within Colab's 12-hour limit
📂 Loaded previous progress from Drive - 1 batches completed
🔥 Device: cuda
GPU: NVIDIA L4
Memory: 23.8 GB
Mixed Precision: Enabled

📊 Loading CIFAR-100 dataset...
✅ CIFAR-100 loaded successfully!
   Training samples: 50,000
   Test samples: 10,000
   Classes: 100
🎯 Total models across all batches: 15
📦 Organized into 7 batches
🎯 Running specific batch: 6 (Swin Tiny Retry)

🏗️ BATCH 6: SWIN TINY RETRY
📋 Models in this batch: swin_tiny

🔧 Training swin_tiny (1/1)
--------------------------------------------------
⚙️ Config: LR=0.0005, BS=16, Epochs=25


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

📊 Model size: 27,596,254 total parameters, 27,596,254 trainable
✅ CIFAR-100 loaded successfully!
   Training samples: 50,000
   Test samples: 10,000
   Classes: 100
📁 Drive directories created at: /content/drive/MyDrive/CIFAR100_Experiment
🚀 Training swin_tiny on cuda
Mixed Precision: True
💾 Checkpoints will be saved to: /content/drive/MyDrive/CIFAR100_Experiment/models


Evaluating: 100%|██████████| 625/625 [00:12<00:00, 50.73it/s]


Epoch [1/25]
Train Loss: 4.5967, Train Acc: 0.0137
Test Loss: 4.6151, Test Acc: 0.0100
Best Acc: 0.0100, Patience: 0/10
💾 Best model saved: /content/drive/MyDrive/CIFAR100_Experiment/models/swin_tiny_best.pth
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/swin_tiny_checkpoint.pth
------------------------------------------------------------


...

Evaluating: 100%|██████████| 625/625 [00:12<00:00, 50.13it/s]


Epoch [13/25]
Train Loss: 4.6064, Train Acc: 0.0098
Test Loss: 4.6070, Test Acc: 0.0100
Best Acc: 0.0391, Patience: 9/10
------------------------------------------------------------


Evaluating: 100%|██████████| 625/625 [00:12<00:00, 50.27it/s]


Epoch [14/25]
Train Loss: 4.6062, Train Acc: 0.0097
Test Loss: 4.6057, Test Acc: 0.0100
Best Acc: 0.0391, Patience: 10/10
------------------------------------------------------------
🛑 Early stopping triggered after 14 epochs
💾 Checkpoint saved: /content/drive/MyDrive/CIFAR100_Experiment/models/swin_tiny_checkpoint.pth
✅ swin_tiny completed successfully!
📊 Final Test Accuracy: 3.91%
⏱️ Training Time: 55.7 minutes
📁 Drive directories created at: /content/drive/MyDrive/CIFAR100_Experiment
💾 Progress saved to Drive - Batch 6 completed
📍 Location: /content/drive/MyDrive/CIFAR100_Experiment/progress/experiment_progress.json

📊 BATCH 6 SUMMARY:
✅ Successfully trained: 1/1 models
⏱️ Batch time: 55.8 minutes
🏆 Best in batch: swin_tiny (3.91%)

📈 Generating Intermediate Results...
✅ Total models completed so far: 2

📊 COMPREHENSIVE RESULTS SUMMARY:
             Model Test Accuracy (%) Training Time (min) Efficiency (Acc%/min) Precision Recall F1-Score
Efficientnet Lite0             83.14       

,Model,Test Accuracy (%),Training Time (min),Efficiency (Acc%/min),Precision,Recall,F1-Score
0,Efficientnet Lite0,83.14,0.0,27225.06,N/A,N/A,N/A
1,Swin Tiny,3.91,55.7,0.07,N/A,N/A,N/A


### Final Experiment Results

In [26]:
import json
import pandas as pd

# Path to your JSON file
json_path = "/content/drive/MyDrive/CIFAR100_Experiment/results/FINAL_experiment_results.json"

# Load JSON
with open(json_path, "r") as f:
    data = json.load(f)

# Extract relevant metrics into a list of dictionaries
summary_data = []
for model_id, metrics in data.items():
    summary_data.append({
        "Model": metrics["model_name"],
        "Final Test Accuracy": metrics["final_test_acc"],
        "Training Time (s)": round(metrics["training_time"], 2),
        "Total Parameters": metrics["total_parameters"],
        "Trainable Parameters": metrics["trainable_parameters"]
    })


df = pd.DataFrame(summary_data)

# Sort by accuracy descending
df = df.sort_values(by="Final Test Accuracy", ascending=False).reset_index(drop=True)

In [27]:
df

,Model,Final Test Accuracy,Training Time (s),Total Parameters,Trainable Parameters
0,efficientnet_b2,0.8548,3756.20,7841894,7841894
1,efficientnet_b0,0.8460,2838.81,4135648,4135648
2,efficientnet_lite0,0.8314,2770.50,3499108,3499108
3,resnet50,0.8290,3794.19,23712932,23712932
4,regnet_y_400mf,0.8195,2799.11,3947244,3947244
5,mobilenet_v3_small,0.7935,2797.01,1620356,1620356
6,resnet18,0.7856,2831.26,11227812,11227812
7,deit_small,0.6421,3440.14,21704164,21704164
8,vit_base_16,0.5479,4666.67,85875556,85875556
9,simple_cnn,0.3895,4470.09,3356324,3356324
